# 2015

**AlexNet** va esdevenir un punt d'inflexió en la visió per computador i la IA. A partir d'aquest punt les xarxes convolucionals es convertiren en l'estat de l'art. Les xarxes cada cop eren més profundes i aquest fest es va convertir en un problema.

Durant l'any 2015 es varen presentar tres articles que suposaren un nou avenç en el nostre camp. Dues noves arquitectures i una tècnica de regularització:

- Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift - google  [enllaç](https://arxiv.org/pdf/1502.03167) 
- Very deep convolutional networks for large-scale image recognition - Oxford [enllaç](https://arxiv.org/pdf/1409.1556) - [video presentació](https://www.youtube.com/watch?v=j1jIoHN3m0s&t=31s)
- Deep Residual Learning for Image Recognition - Microsoft [enllaç](https://arxiv.org/pdf/1512.03385)
  
En la sessió d'avui aprofundirem en aquests conceptes partint del que ja coneixem.

## Batch Normalization

Batch Norm és tècnica que s'aplica en una capa de xarxa neuronal i que s'utilitza habitualment en moltes arquitectures. Sovint s'afegeix com a part d'un bloc convolucional i ajuda a estabilitzar la xarxa durant l'entrenament.

Quan introduim dades a un model d'aprenentatge profund, és una pràctica habitual normalitzar les dades a la mitjana 0 i la variància 1. 

**Per què ho fem?**

Mirem un exemple amb només dues característiques que es troben a escales molt diferents. Com que la sortida de la xarxa és una combinació lineal de cada vector de característiques, això significa que la xarxa aprèn pesos per a cada característica que també es troben a escales diferents. En cas contrari, la característica gran simplement tendrá molta més importància que la característica petita.

Durant el descens del gradient, en el càlcul que es realitza per minimitzar la pèrdua, la xarxa hauria de fer una gran actualització d'un pes en comparació amb l'altre. Això pot fer que la trajectòria de descens del gradient oscil·li cap endavant i cap enrere al llarg d'una dimensió, fent més passos dels que presumiblement serien necessaris per arribar al mínim.

![BatchNORM](img/batch_norm_problem.png "Batch Norm Problem")

En aquest cas, el gràfic de la funció pèrdua sembla un barranc estret. Si descomponem el gradient al llarg de les dues dimensions: És empinat en una dimensió i molt més suau en l'altra. El que succeeix és que  acabem fent una actualització més gran a un pes a causa del seu gran gradient. Això fa que el descens del pendent *reboti* a l'altra direcció. D'altra banda, el gradient més petit al llarg de la segona direcció fa que fem actualitzacions de pes més petites i, per tant, fem passos més petits. Aquesta trajectòria desigual fa que la xarxa tardi més a convergir. En canvi, si les característiques estan a la mateixa escala, el gràfic de la funció de pèrdua és més uniforme. Aleshores, el descens de gradient pot continuar sense problemes fins al mínim local.


### Batch norm a la pràctica

Considerem qualsevol de les capes ocultes d'una xarxa. Les activacions de la capa anterior són simplement les entrades a aquesta capa. La mateixa lògica que ens duu a normalitzar l'entrada de la primera capa també s'aplicarà a cadascuna de les capes ocultes.

Batch Norm és només una altra capa de xarxa que s'insereix entre una capa oculta i la següent capa oculta. La seva funció és agafar les sortides de la primera capa oculta i normalitzar-les abans de passar-les com a entrada de la següent capa oculta.

Una capa Batch Norm també té paràmetres propis:

 - Dos paràmetres que es poden aprendre anomenats **beta** i **gamma**.
 - Dos paràmetres que no es poden aprendre (**mitjana mòbil**  i **variància mòbil**) es guarden com a part de l'"estat" de la capa.

![BatchNORM1](img/batch_norm_aprenentatge.png "Batch Norm Configuració")


De la imatge anterior podem veure:

1. **Activacions**: Les activacions de la capa anterior es passen com a entrada. Hi ha un vector d'activació per a cada característica de les dades.
2. **Calcul de la mitjana i la variància**: Per a cada vector d'activació per separat, calculem la mitjana i la variància de tots els valors del batch.
3. **Normalització**: Calculem els valors normalitzats per a cada vector de característiques d'activació utilitzant la mitjana i la variància corresponents. Aquests valors normalitzats ara tenen una mitjana zero i una variància unitària.
4. **Escalat i desplaçament**: Aquest pas és la gran innovació introduïda per Batch Norm. A diferència de la capa d'entrada, que requereix que tots els valors normalitzats tinguin una mitjana zero i una variància unitària, Batch Norm permet canviar els seus valors (a una mitjana diferent) i escalar (a una variància diferent). Ho fa multiplicant els valors normalitzats per un factor, **gamma**, i afegint-hi un factor, **beta**; Aquesta és una multiplicació element a element, no una multiplicació matricial. El que fa que aquesta innovació sigui interessant és que aquests valors no són hiperparàmetres sinó que són paràmetres entrenables que s'aprenen. En altres paraules, cada capa Batch Norm és capaç de trobar de manera òptima els millors factors per si mateixa i, per tant, pot canviar i escalar els valors normalitzats per obtenir les millors prediccions.
5. **Mitjana mòbil**: A més, Batch Norm també manté un recompte de la mitjana mòbil exponencial (EMA) de la mitjana i la variància. Durant l'entrenament, simplement es calcula aquest EMA però no fa res amb ell. Al final de l'entrenament, simplement desa aquest valor com a part de l'estat de la capa, per utilitzar-lo durant la fase d'inferència.

### Batch norm a la inferència

Durant l'inferència s'utilitzen els dos paràmetres de mitjana mòbil que s'han calculat durant l'entrenament i que estan guardats dins la capa. 

Idealment, durant l'entrenament, podríem haver calculat i desat la mitjana i la variància per a les dades completes; Però això seria molt costós, ja que hauríem de mantenir els valors del conjunt de dades complet a la memòria durant l'entrenament. En canvi, la mitjana mòbil actua com una bona aproximació per a la mitjana i la variància de les dades. És molt més eficient perquè el càlcul és incremental: només hem de recordar la mitjana mòbil més recent.

![BatchNORM2](img/batch_norm_inferencia.png "Batch Norm Configuració")


### Conclusions

El Batch norm és una tècnica de regularització molt emprada, que pot substituir al *droput* sobretot a la part convolucional de la xarxa.
El benefici que proporciona Batch Norm és permetre que el model convergeixi més ràpidament i acceleri l'entrenament. Fa que l'entrenament sigui menys sensible a com s'inicien els pesos i a l'ajustament precís dels hiperparàmetres.

**Batch Norm permet utilitzar ratis d'aprenentatge més alts**. Sense Batch Norm, les taxes d'aprenentatge s'han de mantenir petites per evitar que els grans gradients afectin el descens del gradient.

Batch Norm no funciona bé amb mides batch petites. Això provoca massa renou en la mitjana i la variació de cada mini-lot.


### Batch norm a la pràctica
La capa a `Pytorch` tè la següent cridada, on l'únic paràmetre que és obligatòri és `num_features` que vé determinat per l'arquitectura de la xarxa:
`torch.nn.BatchNorm2d(num_features)`. 

En quant al seu posicionament, hi ha dues opinions sobre on s'ha de col·locar aquesta capa a l'arquitectura: abans o després de l'activació; En l'article  original es col·loca abans.

[Enllaç a la documentació](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)

**Bibliografia**

Per realitzar aquesta secció s'han consultat i emprat diversos recursos:

- Article original.
- Batch Norm Explained Visually — How it works, and why neural networks need it [enllaç](https://towardsdatascience.com/batch-norm-explained-visually-how-it-works-and-why-neural-networks-need-it-b18919692739)
- Batch Norm Explained Visually — Why does it work? [enllaç](https://towardsdatascience.com/batch-norm-explained-visually-why-does-it-work-90b98bcc58a0)

 
---

## Very deep convolutional networks for large-scale image recognition

**VGGNet** (_Visual Geometry Group_) neix de la necessitat de reduir el nombre de paràmetres a les capes convolucionals i millorar el temps d'entrenament de les xarxes existents en aquell moment de la història. En l'article original es presenten dues variants: VGG16 i VGG19 que només es diferencien pel nombre total de capes de la xarxa. 

### Arquitectura

L'entrada a les VGG  és una imatge de mida fixa de 224×224x3, es a dir, és RGB. L'únic pre-processament que es realitza és restar el valor RGB mitjà, calculat al conjunt d'entrenament, de cada píxel.

Traduïnt de l'article:

> La imatge es fa passar per una succesió de capes convolucionals, on utilitzem filtres mida de: 3×3 (que és la mida més petita per capturar la noció d'esquerra/dreta, amunt/avall, centre). En una de les configuracions també utilitzem filtres de convolució 1×1, que es poden veure com una transformació lineal dels canals d'entrada (seguida de la no linealitat ReLU). El pas de convolució (stride) es fixa a 1 píxel; i el padding és de 1 pixel de tal manera que la resolució espacial es conserva després de la convolució. L'agrupació espacial es realitza mitjançant cinc capes de max pooling, que segueixen algunes de les capes convolucionals (no totes les capes convolucionals van seguides de max pooling). L'agrupació màxima es realitza en una finestra de 2×2 píxels, amb stride de 2.
>

> Les capes convolucionals (que tenen una profunditat diferent en diferents arquitectures) són seguides de tres capes totalment connectades (FC): les dues primeres tenen 4096 canals cadascuna, la tercera realitza una classificació ILSVRC de 1000 categories i, per tant, conté 1000 canals (un per a cada classe). La capa final és la capa soft-max. La configuració de les capes totalment connectades és la mateixa en totes les xarxes. Totes les capes ocultes estan seguides d'una capa equipades amb una capa de rectificació no lineal (ReLU).

![VGG16](img/VGG16_params.png "Paràmetres VGG")


#### Discussió

Una de les novetats més rellevants és que aquesta xarxa presenta *kernels* de mida fixa, a diferencia dels *kernels* de mida variable utilitzats a Alexnet (11x11, 5x5, 3x3). A l'article s'argumenta que la capacitat d'obtenir característiques de *kernels* de mida gran es pot replicar fent ús de múltiples *kernels* més petits.

**Exemple**
- Suposem que tenim una capa d'entrada de mida 5x5x1. 
- La implementació d'una capa convolucional 2D amb `kernel=(5,5)` i `stride=1` donarà lloc a un mapa de característiques de mida 1x1.
- El mateix mapa de característiques de sortida es pot obtenir implementant dues capes convolucionals 2D amb `kernel=(3,3)` i `stride=1`.

De l'article:

> Aleshores, què hem guanyat utilitzant, per exemple, una pila de tres capes convolucionals de 3×3 en lloc d'una única capa de 7× 7? 
    En primer lloc, incorporem tres capes de rectificació no lineals en lloc d'una única, la qual cosa fa que la funció de decisió sigui més discriminatòria. En segon lloc, disminuïm el nombre de paràmetres: suposant que tant l'entrada com la sortida d'una pila de tres capes convolucionals amb un kernel de 3×3 té C canals, la pila es parametritza amb $3(3^2C^2)$ = $27C^2$ pesos; al mateix temps, una única capa convolucional amb un kernel de 7×7 requeriria $7^2C^2$ = $49C^2$ paràmetres, és a dir, un 81% més.

En quant als kernels 1x1:

> La incorporació de capes amb kernel 1×1 és una manera d'augmentar la no linealitat de la funció de decisió sense afectar els camps receptius de les capes convolucionals. Fins i tot encara que en el nostre cas la convolució 1×1 és essencialment una projecció lineal a l'espai de la mateixa dimensionalitat (el nombre de canals d'entrada i sortida és el mateix), la funció de rectificació introdueix una no linealitat addicional.
  

![VGG16](img/VGG.png "VGG")

### Entrenament
- La mida de la imatge és fixa a 224x224x3 pixels.
- La mida del batch es va establir en 256 imatges.
- *Dropout* de $0.5$ per a les dues primeres capes completament connectades.
- La taxa d'aprenentatge (*learning rate*) es va establir inicialment en 10−2, i després es va reduir en un factor de 10 quan la precisió del conjunt de validació va deixar de millorar. En total, la taxa d'aprenentatge es va reduir 3 vegades.
- L'aprenentatge es va aturar després de 370K iteracions (74 èpoques).
- Conjunt de dades: [ILSVRC-2012](https://www.image-net.org/challenges/LSVRC/2012/); El conjunt de dades inclou imatges de 1.000 classes i es divideix en tres conjunts: entrenament (1,3 milions d'imatges), validació (50 mil imatges) i test (100 mil imatges).
- El rendiment de la classificació s'avalua mitjançant dues mesures: l'error 1 i 5.

### Recursos

- Video presentació de la xarxa: [enllaç](https://www.youtube.com/watch?v=j1jIoHN3m0s).

### VGG a Pytorch

Pytorch té la seva propia implementació de la VGG. Al següent [enllaç](https://pytorch.org/vision/stable/models/vgg.html) podeu trobar totes les versions implementades. Per cada arquitectura hi ha la versió amb i sense Batch Normalisation.



In [ ]:
import torchvision.models as models

model_vgg = models.vgg16(weights='IMAGENET1K_V1')

---
## Deep Residual Learning for Image Recognition

Les xarxes residuals (ResNets) han revolucionat el camp de l'aprenentatge profund permetent la construcció de xarxes molt més profundes del que era possible abans de la seva aparició.

A l'article es descriu un dels grans problemes que es trobaven en aquells moments:

> Impulsat per la importància del nombre de capes d'un model (la seva profunditat), sorgeix una pregunta: Construir millors xarxes és tan fàcil com apilar més capes? Un obstacle per respondre aquesta pregunta ha estat el famós problema de desaparició/explosió dels gradients (_vanishing gradients_ en anglés), que dificulta la convergència de la xarxa des del principi de l'entrenament. Aquest problema, tanmateix, s'ha abordat en gran part per les diferents normalitzacions que s'apliquen a les dades: inicialització i _batch norm_.


Quan les xarxes més profundes comencen a convergir, s'exposen a un problema de degradació: així com  la xarxa augmenta la seva profunditat, la precisió es satura i després es degrada ràpidament. Inesperadament, aquesta degradació no és causada per un sobreajustament, i afegir més capes a un model condueix a un error d'entrenament més alt.

### Connexions residuals

La hipótesi de partida d'aquest article és que si afegim més profunditat a una xarxa, el seu rendiment com a mínim hauría de ser el mateix que el que té la xarxa original, ja que seria suficient que els nous blocs de capes aprenguessin la funció identitat, es a dir que la seva sortida fos igual a la seva entrada; **A l'article, mostren experimentalment com aquest fet no s'acompleix.**

![Problema](img/res_net_rendiment.png "Problema")

Per solventar aquest problema es proposa el concepte d'aprenentatge residual on a la sortida de cada bloc de la xarxa a més de la sortida dels seus mapes d'activació rep el mapa d'activació de la sortida del bloc anterior. Tal com es pot veure a la figura següent:

![Solució](img/residual_blocks.png "Solució")

D'aquesta manera la xarxa  facilita aprendre la identitat si "es necesari". 

A l'article es menciona que:

> En casos reals, és poc probable que els mapes d'identitat siguin l'òptim, però la nostra reformulació pot ajudar a condicionar el problema. Si la funció òptima està més a prop d'un mapeig d'identitat que d'un mapeig zero, hauria de ser més fàcil per al solucionador trobar les pertorbacions amb referència a un mapeig d'identitat, que aprendre la funció com a nova.


S'adopta l'aprenentatge residual a cada poques capes apilades. La forma de la funció residual és flexible, els experiments de l'article impliquen una funció que té dues o tres capes, però aplicar-ho a més capes és possible.

En quant a la práctica, una funció residual no és més que una connexió entre capes i una consecuent concatenació dels mapes d'activació. Si les mesures no són coincidents s'aplica una transformació simple a la mida per poder sumar.

Si voleu aprofundir en el significat de les connexions residuals: [article](https://d2l.ai/chapter_convolutional-modern/resnet.html#function-classes).

*NOTA*: en aquest context bloc pot ser entés tant una capa com un conjunt de capes consecutives.

### Arquitectura

A l'article es descriuen dues configuracions de xarxa diferent, que s'inspiren principalment en la filosofia de les xarxes VGG.

#### Xarxa plana

Les capes convolucionals tenen  filtres 3×3 i segueixen dues regles de disseny senzilles:

1. Per a la mateixa mida del mapa de característiques de sortida, les capes tenen el mateix nombre de filtres.
2. Si la mida del mapa de característiques es redueix a la meitat, el nombre de filtres es duplica per preservar la complexitat temporal per capa.
  
Realitzem la baixada de dimensions directament a les capes convolucionals amb un `stride=2`. La xarxa acaba amb una capa d'_average pooling_ i un MLP (totalment connectat) amb una sortida de 1000 neurones i funció d'activació _softmax_. El nombre total de capes és de 34.

#### Xarxa residual

A partir de la xarxa anterior, s'insereixen connexions de drecera (¿_residuals_?) que converteixen la xarxa en la seva versió residual homòloga. Les dreceres d'identitat es poden utilitzar directament quan l'entrada i la sortida són de les mateixes dimensions. 

Quan les dimensions són diferents es consideren dues opcions:

1. La drecera encara realitza el mapeig d'identitat, s'afegeix el _padding_ necessari per fer coincidir les dimensions amb valor zero. Aquesta opció no introdueix cap paràmetre addicional;
2. La drecera es redimensiona per fer coincidir les dimensions.



![Xarxes](img/res_net_comparacio.PNG "xarxes")

### Implementació

- El conjunt de dades és ImageNet
- La mida de la imatge es canvia amb el seu costat més curt de forma aleatòria a [256, 480] per augmentar l'escala.
- S'obté un retall de 224 × 224 aleatòriament a partir d'una imatge o el seu gir horitzontal, amb la mitjana per píxel restada. 
- S'adopta la normalització per lots (_batch normalization_) just després de cada convolució i abans de l'activació (_ReLU_).
- S'utilitza SGD com optimitzador amb una mida de mini-lot de 256.
-  La taxa d'aprenentatge (_learning rate_) comença des de 0,1 i es divideix per 10 quan l'error s'_estanca_, els models estan entrenats per a fins a $60 × 10^4$ iteracions.
- No es fa servir _dropout_.
- Per obtenir els millors resultats, s'adopta la forma totalment convolucional (no MLP) i es fa una mitjana de les puntuacions a diverses escales (les imatges es redimensionen de manera que el costat més curt estigui en el cojunt {224, 256, 384, 480, 640}).

### ResNet a Pytorch

Pytorch té la seva propia implementació de la ResNet que com posa a una nota té alguna diferència amb l'original. Al següent [enllaç](https://pytorch.org/vision/stable/models/resnet.html) podeu trobar les 5 versions implementades que tenen entre 18 i 152 capes.

In [ ]:
import torchvision.models as models

model_res = models.resnet18(weights="IMAGENET1K_V1")

---
### Exercici / Tutorial

Crearem la nostra propia capa residual com a subclasse de [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

In [1]:
import torch
import torch.nn as nn
from torch import Tensor

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        #TODO: Put your code here

    def forward(self, x):

        #TODO: Put your code here

        return x

### EXTRA: Adaptive Average Pooling

L'*Adaptive Average Pooling* (AAP) és un tipus de capa que permet agrupar les dades d'entrada en una sortida de mida fixa, independentment de la mida d'entrada. Això és especialment útil en xarxes que poden tenir diferents arquitectures o entrades de de dades de diferents dimensionalitats.

Com a característiques principals cal comentar:

1. **Mida de sortida fixa**: a diferència de les capes de **pooling** que coneixem fins ara (*max pooling*) que redueixen les dimensions espacials de l'entrada per un factor fix, l'AAP permet especificar la mida exacta de la sortida. Això vol dir que podem assegurar que la sortida de la capa sempre tingui una mida determinada, independentment de la mida d'entrada.
2. **Adaptabilitat**: la capa s'adapta a la mida d'entrada calculant la mida de la finestra d'agrupació de forma dinàmica. Això es fa dividint les dimensions d'entrada per les dimensions de sortida desitjades. La mida de la finestra s'arrodoneix a l'enter més proper per assegurar-se que la mida de sortida coincideix amb les dimensions especificades.
3. **Agrupació mitjana**: la capa realitza una agrupació mitjana, el que significa que pren la mitjana de tots els valors de la finestra. Com ja sabem, aquesta és una operació habitual, ja que ajuda a reduir la dimensionalitat de l'entrada alhora que es preserva les característiques més importants.